In [1]:
import time
import random
import math

In [2]:
pessoas = [("Amanda", "CWB"),
           ("Pedro", "GIG"),
           ("Marcos", "POA"),
           ("Priscila", "FLN"),
           ("Jessica", "CNF"),
           ("Paulo", "GYN"),]

In [3]:
destino = "GRU"

In [4]:
voos = {}

In [5]:
with open("voos.txt", "r") as arq:
    for linha in arq.readlines():
        origem, destino, saida, chegada, preco = linha.split(",")
        voos.setdefault((origem, destino), [])
        voos[(origem, destino)].append((saida, chegada, int(preco)))

In [6]:
# [1,4, 3,2, 7,3, 6,3, 2,4, 5,3 ]

def imprimir_agenda(agenda: list[int]) -> None:
    id_voo = -1
    for i in range(len(agenda)//2):
        nome = pessoas[i][0]
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][agenda[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][agenda[id_voo]]
        print(f"{nome} {origem} {ida[0]} {ida[1]} {ida[2]} {volta[0]} {volta[1]} {volta[2]}\n")

In [7]:
agenda = [1,4, 3,2, 7,3, 6,3, 2,4, 5,3 ]

imprimir_agenda(agenda)

Amanda CWB 8:04 10:11 95 12:08 14:05 142

Pedro GIG 10:30 14:57 290 9:49 13:51 229

Marcos POA 17:08 19:08 262 10:32 13:16 139

Priscila FLN 15:34 18:11 326 11:08 14:38 262

Jessica CNF 9:42 11:32 169 12:08 14:47 231

Paulo GYN 13:37 15:08 250 11:07 13:24 171



In [8]:
def obter_minutos(horaStr: str):
    hora = time.strptime(horaStr, "%H:%M")
    minutos = hora[3] * 60 + hora[4]
    return minutos

In [9]:
def custo(solucao) -> None:
    preco_total = 0
    ultima_chegada = 0
    primeira_partida = obter_minutos("23:59")

    id_voo = -1
    for i in range(len(solucao) // 2):
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][solucao[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][solucao[id_voo]]

        preco_total += ida[2] + volta[2]

        if ultima_chegada < obter_minutos(ida[1]):
            ultima_chegada = obter_minutos(ida[1])

        if primeira_partida > obter_minutos(volta[0]):
            primeira_partida = obter_minutos(volta[0])
    
    total_espera = 0
    id_voo = -1
    for i in range(len(solucao) // 2):
        origem = pessoas[i][1]
        id_voo += 1
        ida = voos[(origem, destino)][solucao[id_voo]]
        id_voo += 1
        volta = voos[(destino, origem)][solucao[id_voo]]

        total_espera += (ultima_chegada - obter_minutos(ida[1])) + (obter_minutos(volta[0]) - primeira_partida)

    if ultima_chegada > primeira_partida:
        preco_total += 50

    return preco_total + total_espera 

In [10]:
custo(agenda)

4635

### PESQUISA RANDÔMICA

In [11]:
def pesquisa_randomica(dominio, funcao_custo):
    melhor_custo = 9999999999
    melhor_solucao = []
    for i in range(0, 1000):
        solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
        custo = funcao_custo(solucao)
        if custo < melhor_custo:
            melhor_custo = custo
            melhor_solucao = solucao
    return melhor_solucao

In [12]:
dominio = [(0, 9)] * (len(pessoas) * 2)
solucao_randomica = pesquisa_randomica(dominio, custo)
custo_randomica = custo(solucao_randomica)
custo_randomica

3688

In [13]:
imprimir_agenda(solucao_randomica)

Amanda CWB 12:34 15:02 109 15:25 16:58 62

Pedro GIG 12:19 15:25 342 19:57 23:15 512

Marcos POA 12:08 14:59 149 15:50 18:45 243

Priscila FLN 14:01 17:24 338 15:23 18:49 150

Jessica CNF 14:22 16:32 126 17:06 20:00 95

Paulo GYN 11:08 13:07 175 16:35 18:56 144



### Subida da Encosta

In [20]:
def subida_da_encosta(dominio, funcao_custo):
    solucao = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
    while True:
        vizinhos = []
        for i in range(len(dominio)):
            if solucao[i] == dominio[i][0]:
                if solucao[i] != dominio[i][1]:
                    vizinhos.append(solucao[0:i] + [solucao[i] + 1] + solucao[i + 1:])
            if solucao[i] < dominio[i][1]:
                if solucao[i] != dominio[i][0]:
                    vizinhos.append(solucao[0:i] + [solucao[i] - 1] + solucao[i + 1:])
            
        atual = funcao_custo(solucao)
        melhor = atual

        for i in range(len(vizinhos)):
            custo = funcao_custo(vizinhos[i])
            if custo < atual:
                melhor = custo
                solucao = vizinhos[i]

        if melhor == atual:
            break
        
    return solucao

In [71]:
dominio = [(0, 9)] * (len(pessoas) * 2)
solucao_subida_da_encosta = subida_da_encosta(dominio, custo)
custo_subida_da_encosta = custo(solucao_subida_da_encosta)
custo_subida_da_encosta

2872

In [72]:
imprimir_agenda(solucao_subida_da_encosta)

Amanda CWB 8:04 10:11 95 9:58 11:18 130

Pedro GIG 6:12 10:22 230 9:49 13:51 229

Marcos POA 8:27 10:45 139 13:37 15:33 142

Priscila FLN 7:34 9:40 324 11:08 14:38 262

Jessica CNF 8:25 10:34 157 10:33 13:11 132

Paulo GYN 7:39 10:24 219 11:07 13:24 171

